In [ ]:
import json
import os
import cv2
from collections import defaultdict
from tqdm.notebook import tqdm
    
v7_file = "v7_night_coco_03_20_2025.json"
new_data_dir = "/shared/users/raajitha/YOLOVexperiments/night_time_data/"
if not os.path.exists(new_data_dir):
    os.makedirs(new_data_dir)
new_json_file = "/shared/users/raajitha/YOLOVexperiments/night_time_data/03_20_train_annotations_coco_fmt.json"
v7_data = json.load(open(v7_file, "r"))

ann_img_ids = [ann["image_id"] for ann in v7_data["annotations"]]
img_ids_not_in_ann_img_ids = [img["id"] for img in v7_data["images"] if img["id"] not in ann_img_ids]

old_category_to_new = {333014825: 0, 2354408709: 1, 3632848955: 2}

new_categories = [{'supercategory': 'none', 'id': 0, 'name': 'aircraft'},
                  {'supercategory': 'none', 'id': 1, 'name': 'aircraft-nav-light'},
                  {'supercategory': 'none', 'id': 2, 'name': 'strobe'}]

cls_id_to_name = {cat["id"]:cat["name"] for cat in new_categories}

categories = []
images = []

img_id = 0
old_img_id_to_new_img_id = {}
for image in tqdm(v7_data["images"]):
    if image["id"] in img_ids_not_in_ann_img_ids:
        continue
    image_file_name = os.path.join("images", image["file_name"])
    old_img_id_to_new_img_id[image["id"]] = img_id
    new_image = {"id": img_id,
                 "width": image["width"], 
                 "height": image["height"],
                 "file_name": image_file_name}
    images.append(new_image)
    img_id += 1
    
annotations = []
ann_id = 0
for ann in tqdm(v7_data["annotations"]):
    annotation = {"id": ann_id,
                 "category_id": old_category_to_new[ann["category_id"]],
                 "iscrowd": False, 
                 "ignore": False,
                 "bbox": ann["bbox"],
                 "area": ann["area"],
                 "image_id": old_img_id_to_new_img_id[ann["image_id"]],
                 "attributes": {"class_name": cls_id_to_name[old_category_to_new[ann["category_id"]]]}}
    annotations.append(annotation)
    ann_id+=1

new_json_data = {"info": v7_data["info"], "categories": new_categories, "images": images, "annotations": annotations}
with open(new_json_file, "w") as f:
    json.dump(new_json_data, f, indent=4)

In [26]:
import json
file = "/shared/users/raajitha/YOLOVexperiments/night_time_data/val_03_20_split_vid_coco_fmt.json"
new_file = "/shared/users/raajitha/YOLOVexperiments/night_time_data/03_20_val_annotations_coco_fmt.json"
data = json.load(open(file, "r"))
data_dir = "/shared/users/raajitha/YOLOVexperiments/night_time_data/"
del data["videos"]
for image in data["images"]:
    image["file_name"] = image["name"].split(data_dir)[-1]
    del image["name"]
    del image["sid"]
    del image["fid"]
for ann in data["annotations"]:
    del ann["sid"]

with open(new_file, "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
for img in new_json_data["images"]:
    filepath = os.path.join(new_data_dir, img["file_name"])
    if not os.path.exists(filepath):
        print(filepath)

In [ ]:
import requests
import os

def download_file(url, filename):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {filename} successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")

url = "https://darwin.v7labs.com/api/v2/teams/zipline/files/2cfa5cc3-515d-454b-a2ab-5ee8d23724dd/sections/0"
filename = os.path.basename("0")
download_file(url, filename)